In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE 
import zipfile
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from lazypredict.Supervised import LazyClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter

In [3]:
nltk.data.path.append('/kaggle/working/nltk_data')

nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.download('omw-1.4', download_dir='/kaggle/working/nltk_data') 
nltk.download('punkt', download_dir='/kaggle/working/nltk_data')  
nltk.download('stopwords', download_dir='/kaggle/working/nltk_data')

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package punkt to /kaggle/working/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
wordnet_path = '/kaggle/working/nltk_data/corpora/wordnet.zip'

with zipfile.ZipFile(wordnet_path, 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/nltk_data/corpora/')

# **Load Dataset**

In [5]:
df = pd.read_csv('/kaggle/input/amazon-alexa-reviews/amazon_alexa.tsv', sep='\t')

print(df.head())

   rating       date         variation  \
0       5  31-Jul-18  Charcoal Fabric    
1       5  31-Jul-18  Charcoal Fabric    
2       4  31-Jul-18    Walnut Finish    
3       5  31-Jul-18  Charcoal Fabric    
4       5  31-Jul-18  Charcoal Fabric    

                                    verified_reviews  feedback  
0                                      Love my Echo!         1  
1                                          Loved it!         1  
2  Sometimes while playing a game, you can answer...         1  
3  I have had a lot of fun with this thing. My 4 ...         1  
4                                              Music         1  


# **Data Cleaning**

In [6]:
df = df.drop_duplicates(subset=['verified_reviews'], keep='first')
df = df.dropna(subset=['verified_reviews', 'feedback'])
df['verified_reviews'] = df['verified_reviews'].astype(str)
df = df.reset_index(drop=True)

# **Data Preprocessing**

In [7]:

def preprocess_text(text):

    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()

    # Remove stopwords and lemmatize
    text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
    return ' '.join(text)

In [8]:
df['verified_reviews'] = df['verified_reviews'].fillna("").astype(str)
df['cleaned_text'] = df['verified_reviews'].apply(preprocess_text)

# **Spliting Data**

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'], df['feedback'], test_size=0.2, random_state=42
)

# **TF-IDF**

In [10]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# **SMOTE Upsampling**

In [11]:
smote = SMOTE(random_state=42)
X_train_tfidf_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)

In [12]:
Counter(y_train_balanced)

Counter({0: 1667, 1: 1667})

# **DecisionTreeClassifier**

In [13]:
classifier = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=30,
    max_features=None,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42,
    splitter='best'
)
classifier.fit(X_train_tfidf_balanced, y_train_balanced)

DecisionTreeClassifier(criterion='entropy', max_depth=30, random_state=42)

In [14]:
y_pred = classifier.predict(X_test_tfidf)

In [15]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9043478260869565
Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.19      0.21        32
           1       0.94      0.96      0.95       428

    accuracy                           0.90       460
   macro avg       0.60      0.57      0.58       460
weighted avg       0.89      0.90      0.90       460



# **Save Model & Vectorizer**

In [17]:
import pickle

with open('decision_tree_model.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!
